In [1]:
import os
import json
import dotenv
import textwrap
import svgwrite
from openai import OpenAI
from PyPDF2 import PdfReader

dotenv.load_dotenv()

def extract_brand_guidelines(pdf_path):
    """Extract text from brand guidelines PDF"""
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

def analyze_brand_with_ai(text):
    """Analyze brand guidelines and extract structured rules"""
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    analysis_prompt = textwrap.dedent("""
    Analyze the brand guidelines and create structured rules for asset generation.
    Focus on these aspects:
    
    1. Color System:
    - Primary palette (HEX/RGB)
    - Secondary palette
    - Usage ratios
    - Color combinations to avoid
    
    2. Typography:
    - Primary typeface (headings)
    - Secondary typeface (body)
    - Tertiary typeface (accent)
    - Size hierarchy
    - Line spacing rules
    
    3. Imagery Rules (if mentioned):
    - Graphic style description (geometric/organic/abstract)
    - Shape vocabulary
    - Line thickness
    - Pattern styles
    - Iconography rules
    - Composition principles
    
    4. Brand Graphics (if described):
    - Recurring graphic elements
    - Spatial relationships
    - Scaling rules
    - Opacity guidelines
    - Texture usage
    
    5. Composition:
    - Grid systems
    - Margins/padding rules
    - Alignment principles
    - White space usage
    
    Format as JSON with these keys:
    {
        "color_system": { ... },
        "typography": { ... },
        "imagery_rules": { ... },
        "brand_graphics": { ... },
        "composition_rules": { ... }
    }
    """)
    
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{
            "role": "user",
            "content": f"{analysis_prompt}\n\nBRAND GUIDELINES:\n{text}"
        }],
        response_format={"type": "json_object"}
    )
    
    return json.loads(response.choices[0].message.content)

def generate_svg_design(brand_specs, output_path):
    """Generate SVG design based on brand specifications"""
    dwg = svgwrite.Drawing(output_path, profile='tiny', size=("800px", "600px"))
    
    # Background
    dwg.add(dwg.rect(insert=(0, 0), size=("100%", "100%"), fill=brand_specs.get("secondary_color", "#FFFFFF")))
    
    # Header with primary color
    dwg.add(dwg.rect(insert=(0, 0), size=("100%", "80px"), fill=brand_specs.get("primary_color", "#000000")))
    
    # Sample logo text
    dwg.add(dwg.text("Brand Logo", insert=(20, 50), 
                    font_family=brand_specs.get("primary_font", "Arial"),
                    fill=brand_specs.get("secondary_color", "#000000"),
                    font_size="40px"))
    
    # Content area
    content = dwg.add(dwg.g(transform="translate(50, 100)"))
    
    # Sample social media post
    content.add(dwg.text("Social Media Post", insert=(0, 30),
                        font_family=brand_specs.get("secondary_font", "Helvetica"),
                        fill=brand_specs.get("primary_color", "#000000"),
                        font_size="24px"))
    
    # Save SVG
    dwg.save()

# Example usage
if __name__ == "__main__":
    # Step 1: Extract text from PDF
    brand_text = extract_brand_guidelines("ripen-brandbook.pdf")
    
    # Step 2: Analyze with AI
    brand_rules = analyze_brand_with_ai(brand_text)
    
    print("Structured Brand Rules:")
    print(json.dumps(brand_rules, indent=2))
    
    # Step 3: Generate SVG
    generate_svg_design({
        "primary_color": "#2A5CAA",
        "secondary_color": "#FFFFFF",
        "primary_font": "Helvetica",
        "secondary_font": "Arial"
    }, "social_media_post.svg")

Structured Brand Rules:
{
  "color_system": {
    "primary_palette": [
      {
        "name": "Celtic Blue",
        "HEX": "#275BE6",
        "RGB": "39, 91, 230"
      },
      {
        "name": "Razzle Rose",
        "HEX": "#EE30CE",
        "RGB": "238, 48, 206"
      },
      {
        "name": "Ripen Mustard",
        "HEX": "#FADE55",
        "RGB": "250, 222, 85"
      },
      {
        "name": "Ripen Pumpkin",
        "HEX": "#FF7E1D",
        "RGB": "255, 126, 29"
      },
      {
        "name": "Carmine Pink",
        "HEX": "#EB3737",
        "RGB": "235, 55, 55"
      },
      {
        "name": "Summer Green",
        "HEX": "#5A9470",
        "RGB": "90, 148, 112"
      },
      {
        "name": "Ripen Jet",
        "HEX": "#353535",
        "RGB": "53, 53, 53"
      },
      {
        "name": "Simply White",
        "HEX": "#FFFFFF",
        "RGB": "255, 255, 255"
      }
    ],
    "secondary_palette": [],
    "usage_ratios": "Colors can be mixed and matched across 

In [2]:
def generate_svg_from_prompt(prompt, brand_rules, output_path):
    """Generate SVG design based on natural language prompt and brand rules"""
    # 1. Parse the prompt using AI
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    design_prompt = f"""
    Based on this design request: "{prompt}"
    
    Create an SVG design that follows these brand guidelines:
    {json.dumps(brand_rules, indent=2)}
    
    Return a JSON object with these keys:
    - background: color to use
    - elements: array of objects with:
      - type: "rect", "text", "circle", "path", etc.
      - properties: position, size, color, etc.
      - content: text content if applicable
    """
    
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{"role": "user", "content": design_prompt}],
        response_format={"type": "json_object"}
    )
    
    # 2. Parse the AI response
    design_spec = json.loads(response.choices[0].message.content)
    
    # 3. Generate SVG from the specification
    dwg = svgwrite.Drawing(output_path, profile='tiny', size=("800px", "600px"))
    
    # Add background
    dwg.add(dwg.rect(insert=(0, 0), size=("100%", "100%"), 
                    fill=design_spec.get("background", "#FFFFFF")))
    
    # Add each element from the specification
    for element in design_spec.get("elements", []):
        if element["type"] == "rect":
            dwg.add(dwg.rect(
                insert=(element["properties"].get("x", 0), 
                        element["properties"].get("y", 0)),
                size=(element["properties"].get("width", "100px"), 
                      element["properties"].get("height", "100px")),
                fill=element["properties"].get("fill", "#000000")
            ))
        elif element["type"] == "text":
            dwg.add(dwg.text(
                element.get("content", ""),
                insert=(element["properties"].get("x", 0), 
                        element["properties"].get("y", 0)),
                font_family=element["properties"].get("font_family", "Arial"),
                fill=element["properties"].get("fill", "#000000"),
                font_size=element["properties"].get("font_size", "16px")
            ))
        # Add more element types as needed
    
    # Save SVG
    dwg.save()
    return output_path

In [3]:
# Example prompts:
prompts = [
    "Create an Instagram post announcing our summer sale with a beach theme",
    "Design a LinkedIn banner showcasing our company values with abstract shapes",
    "Make a Twitter post with our logo centered and a quote about innovation"
]

# Generate designs from prompts
for i, prompt in enumerate(prompts):
    generate_svg_from_prompt(
        prompt, 
        brand_rules,  # Your analyzed brand rules from earlier
        f"design_{i}.svg"
    )

KeyboardInterrupt: 